In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import sys
sys.path.append("./../")

In [54]:
# default libraries
import time
import datetime
import pickle
import pprint
from pathlib import Path
import math


# third party libraries
import torch
import torchvision
import torchvision.transforms as T

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from dotted_dict import DottedDict


Files

In [55]:
# path of experiments
p_data = Path('/mnt/data/experiments/EfficientCN/sem_comp')

In [71]:
# trained with marging loss generated semantic compactness
p_cnn_cr = p_data / 'semcomp_mnist_MnistCNN_CR_2022_02_03_16_27_32'
p_cnn_cr_sf = p_data / 'semcomp_mnist_MnistCNN_CR_SF_2022_02_03_16_27_48'
p_cnn_r = p_data / 'semcomp_mnist_MnistCNN_R_2022_02_04_09_41_08'
p_effcn = p_data / 'semcomp_mnist_MnistEffCapsNet_2022_02_03_17_49_52'
p_cn = p_data / 'semcomp_mnist_CapsNet_2022_03_07_13_51_57'

In [72]:
# trained without reconstruction loss generated semantic compactness
p_cnn_cr_no = p_data / 'semcomp_mnist_MnistCNN_CR_2022_02_07_11_05_35'
p_cnn_cr_sf_no = p_data / 'semcomp_mnist_MnistCNN_CR_SF_2022_02_07_11_05_17'
p_cnn_r_no = p_data / 'semcomp_mnist_MnistCNN_R_2022_02_07_11_14_17'
p_effcn_no = p_data / 'semcomp_mnist_MnistEffCapsNet_2022_02_07_11_04_29'
p_cn_no = p_data / 'semcomp_mnist_CapsNet_2022_03_08_13_26_27'

In [75]:
def load_exp(p_exps):
    kld = []
    pca = []

    for p_exp in p_exps:

        file1 = open(p_exp / 'stats_affine_xtrans.pkl', 'rb')
        xtrans = DottedDict(pickle.load(file1))
        file1.close()

        file2 = open(p_exp / 'stats_affine_ytrans.pkl', 'rb')
        ytrans = DottedDict(pickle.load(file2))
        file2.close()

        file3 = open(p_exp / 'stats_affine_rot.pkl', 'rb')
        rot = DottedDict(pickle.load(file3))
        file3.close()

        file4 = open(p_exp / 'stats_affine_scale.pkl', 'rb')
        scale = DottedDict(pickle.load(file4))
        file4.close()

        file5 = open(p_exp / 'stats_affine_shear.pkl', 'rb')
        shear = DottedDict(pickle.load(file5))
        file5.close()

        file6 = open(p_exp / 'config.pkl', 'rb')
        config = DottedDict(pickle.load(file6))
        file6.close()


        kld_mean = [xtrans.kld.kld_mean, ytrans.kld.kld_mean, rot.kld.kld_mean, scale.kld.kld_mean, shear.kld.kld_mean]
        kld.append(kld_mean)
        pca_mean = [xtrans.pca.pca_mean.tolist(), ytrans.pca.pca_mean.tolist(), rot.pca.pca_mean.tolist(), scale.pca.pca_mean.tolist(), shear.pca.pca_mean.tolist()]
        pca.append(pca_mean)

    return kld, pca

In [76]:
p_exps = [p_cnn_r, p_cnn_cr, p_cnn_cr_sf, p_effcn, p_cn]
kld, pca = load_exp(p_exps)

p_exps = [p_cnn_r_no, p_cnn_cr_no, p_cnn_cr_sf_no, p_effcn_no, p_cn_no]
kld_no, pca_no = load_exp(p_exps)

In [81]:
def matplot(matrix, x_ticks=None, y_ticks=None, title=None):
    fig, ax = plt.subplots(figsize=[8,8])
    im = ax.imshow(matrix, cmap='copper')

    # Show all ticks and label them with the respective list entries
    if x_ticks != None:
        ax.set_xticks(np.arange(len(x_ticks)), labels=x_ticks)
        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                rotation_mode="anchor")
    if y_ticks != None:
        ax.set_yticks(np.arange(len(y_ticks)), labels=y_ticks)

    if title != None:
        ax.set_title(title)

    # Loop over data dimensions and create text annotations.
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            text = ax.text(j, i, matrix[i, j],
                        ha="center", va="center", color="w")

    fig.tight_layout()
    plt.savefig('/home/mkoch/projects/EfficientCN/results/kld.png')

    plt.show()

In [ ]:
affines = ["x trans", "y trans", "rot", "scale","shear"]
models = ["CNN-CR", "CNN-CR-SF", "EffCN", "CN", "CNN-CR no rec", "CNN-CR-SF no rec", "EffCN no rec", "CN  no rec"] 
title = "KL Divergence on MNIST on effcn and cnn models"

a = torch.tensor(kld[1:])
b = torch.tensor(kld_no[1:])

kld_c = torch.cat((a,b), dim=0).numpy()


kld_c

matplot(kld_c, affines, models, title)


Datasets MNIST
Factors Rotation Trans-X Trans-Y Scale Shear-X Shear-Y
ConvNet-R 0.0003 0.0016 0.0009 0.0004 0.0003 0.0007
ConvNet-CR 0.0028 0.0038 0.0032 0.0052 0.0058 0.0022
ConvNet-CR-SF 0.0325 0.2010 0.3192 0.0146 0.0476 0.0506
CapsNet 0.0031 0.0107 0.0464 0.0026 0.0098 0.0021

In [ ]:
affines = ["x_trans", "y_trans", "rot", "scale","shear"]
models = ["CNN-CR", "CNN-CR-SF", "EffCN", "CN", "CNN-CR no rec", "CNN-CR-SF no rec", "EffCN no rec", "CN  no rec"] 
title = 'Mean Eigenvals of u_h caps'

a = torch.tensor(pca[1:])
b = torch.tensor(pca_no[1:])
pca_c = torch.cat((a,b), dim=0).numpy()
x = torch.arange(1,pca_c.shape[2]+1,1)


fig, ax = plt.subplots(1,pca_c.shape[1], figsize=[20,4], sharey=True)

id = 0
aff = 0
ax[id].plot(x, pca_c[0,aff], 'b*',label=models[0])
ax[id].plot(x, pca_c[1,aff], 'r*',label=models[1])
ax[id].plot(x, pca_c[2,aff], 'g*',label=models[2])
ax[id].plot(x, pca_c[3,aff], 'c*',label=models[3])
ax[id].plot(x, pca_c[4,aff], 'b.',label=models[4])
ax[id].plot(x, pca_c[5,aff], 'r.',label=models[5])
ax[id].plot(x, pca_c[6,aff], 'g.',label=models[6])
ax[id].plot(x, pca_c[7,aff], 'c.',label=models[7])
ax[id].set_title(affines[0])
ax[id].legend()
id += 1
aff += 1
ax[id].plot(x, pca_c[0,aff], 'b*',label=models[0])
ax[id].plot(x, pca_c[1,aff], 'r*',label=models[1])
ax[id].plot(x, pca_c[2,aff], 'g*',label=models[2])
ax[id].plot(x, pca_c[3,aff], 'c*',label=models[3])
ax[id].plot(x, pca_c[4,aff], 'b.',label=models[4])
ax[id].plot(x, pca_c[5,aff], 'r.',label=models[5])
ax[id].plot(x, pca_c[6,aff], 'g.',label=models[6])
ax[id].plot(x, pca_c[7,aff], 'c.',label=models[7])
ax[id].set_title(affines[1])
ax[id].legend()
id += 1
aff += 1
ax[id].plot(x, pca_c[0,aff], 'b*',label=models[0])
ax[id].plot(x, pca_c[1,aff], 'r*',label=models[1])
ax[id].plot(x, pca_c[2,aff], 'g*',label=models[2])
ax[id].plot(x, pca_c[3,aff], 'c*',label=models[3])
ax[id].plot(x, pca_c[4,aff], 'b.',label=models[4])
ax[id].plot(x, pca_c[5,aff], 'r.',label=models[5])
ax[id].plot(x, pca_c[6,aff], 'g.',label=models[6])
ax[id].plot(x, pca_c[7,aff], 'c.',label=models[7])
ax[id].set_title(affines[2])
ax[id].legend()
id += 1
aff += 1
ax[id].plot(x, pca_c[0,aff], 'b*',label=models[0])
ax[id].plot(x, pca_c[1,aff], 'r*',label=models[1])
ax[id].plot(x, pca_c[2,aff], 'g*',label=models[2])
ax[id].plot(x, pca_c[3,aff], 'c*',label=models[3])
ax[id].plot(x, pca_c[4,aff], 'b.',label=models[4])
ax[id].plot(x, pca_c[5,aff], 'r.',label=models[5])
ax[id].plot(x, pca_c[6,aff], 'g.',label=models[6])
ax[id].plot(x, pca_c[7,aff], 'c.',label=models[7])
ax[id].set_title(affines[3])
ax[id].legend()
id += 1
aff += 1
ax[id].plot(x, pca_c[0,aff], 'b*',label=models[0])
ax[id].plot(x, pca_c[1,aff], 'r*',label=models[1])
ax[id].plot(x, pca_c[2,aff], 'g*',label=models[2])
ax[id].plot(x, pca_c[3,aff], 'c*',label=models[3])
ax[id].plot(x, pca_c[4,aff], 'b.',label=models[4])
ax[id].plot(x, pca_c[5,aff], 'r.',label=models[5])
ax[id].plot(x, pca_c[6,aff], 'g.',label=models[6])
ax[id].plot(x, pca_c[7,aff], 'c.',label=models[7])
ax[id].set_title(affines[4])
ax[id].legend()


fig.suptitle('Mean Eigenvals of u_h caps')
fig.tight_layout()
plt.savefig('/home/mkoch/projects/EfficientCN/results/pca.png')
plt.show()